In [1]:
from collections import Counter, defaultdict
import json
import pdb

import numpy as np
import pandas as pd

# Filter

In [2]:
# with open("/data/wei/hpa-webdataset-all-composite/HPACombineDatasetInfo.pickle", "rb") as in_file:
#     data_info = pickle.load(in_file)
# len(data_info), data_info[0]
metadata = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv", index_col=0)
metadata

/tmp/ipykernel_51865/1073944846.py:4: DtypeWarning: Columns (8,9,15,17,19,20,34,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv", index_col=0)


,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,image_id,ImageHeight,ImageWidth,imputation_split,protcl_denovo_split,prot_denovo_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,7.0,NaN,1_A1_1,1728.0,1728.0,train,train,test
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,6.0,NaN,1_A1_2,1728.0,1728.0,train,train,test
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,5.0,NaN,1_A10_1,NaN,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,4.0,NaN,1_A10_2,NaN,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,NaN,3.0,NaN,1_A11_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_20,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_21,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_22,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_23,NaN,NaN,NaN,NaN,NaN


In [3]:
cell_masks_metadata = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-cells.csv", index_col=0)
cell_masks_metadata

,ID,ImageHeight,ImageWidth,maskid,cellmask,hpa_index
0,1657_F11_3,2048,2048,1,eNqlkMEKwjAMQH9pTarDo7eJadbhVgSZXoa4reL/32wo9S...,373348
1,1657_F11_3,2048,2048,2,eNp9lOlu2zoQhV9JQzlpcIO2QYsEt4k59CbL2ijKlmTL6/...,373348
2,1657_F11_3,2048,2048,3,eNqVleuSozYQhV+Jg12TyrUqqSRVmQyNwVcuAnERtjGG9/...,373348
3,1657_F11_3,2048,2048,4,eNqlUtsOgjAM/aV1gBJMjA9qIowWLw8makK8JD6g/P+bG3...,373348
4,1657_F11_3,2048,2048,5,eNqFVNGSqjgQ/aWc4O4U1D7sw0rtXCeNCCMiCQGCCir4/2...,373348
...,...,...,...,...,...,...
1800525,1874_C11_45,3072,3072,30,eNqVVG2P2yAM/kvYuZ20L5P2ZdK02F76HnrrO2nVNuv//z...,676417
1800526,1874_C11_45,3072,3072,31,eNrFU9lOwzAQ/CV7clQIiQcEiDbZhEs4bSWaNgi7SMD/vx...,676417
1800527,1874_C11_45,3072,3072,32,eNqNUNFOwzAM/KVwg8Ab4gmlmpu1CRvdmodlG6xNsvz/Gz...,676417
1800528,1874_C11_45,3072,3072,33,eNqtUstuwjAQ/KV4QkECqVUPINFqSHg1hTjOw4Y0BDX/f6...,676417


In [7]:
metadata["image_id"]

0              1_A1_1
1              1_A1_2
2             1_A10_1
3             1_A10_2
4             1_A11_1
              ...    
1086885    2184_H8_20
1086886    2184_H8_21
1086887    2184_H8_22
1086888    2184_H8_23
1086889    2184_H8_24
Name: image_id, Length: 1086890, dtype: object

In [9]:
(cell_masks_metadata["ID"] == "1702_E1_13").sum(), (metadata["image_id"]  == "1702_E1_13").sum()

(34, 1)

In [6]:
# filter_func = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x['locations']) != "nan"
# filter_func2 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED'
# filter_func3 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x["gene_names"]) != "nan"
# # filter_func4 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x["gene_names"]) != "nan" and x["sequences"]
# flt_data_info = list(filter(filter_func, data_info))
# flt2_data_info = list(filter(filter_func2, data_info))
# flt3_data_info = list(filter(filter_func3, data_info))
# # flt4_data_info = list(filter(filter_func4, data_info))
# len(flt_data_info), len(flt2_data_info), len(flt3_data_info)

hpa_idcs_w_masks = set(cell_masks_metadata["hpa_index"])
v23_metadata = metadata[(metadata["latest_version"] == 23) & (metadata["Ab state"] == "IF_FINISHED") & metadata.index.isin(hpa_idcs_w_masks)]
len(v23_metadata), len(set(hpa_idcs_w_masks) & set(v23_metadata.index))

(83445, 83445)

In [7]:
# flt3_metadata = metadata[(metadata["status"] == 35) & (metadata["Ab state"] == "IF_FINISHED") & (metadata["gene_names"].astype(str) != "nan") & metadata.index.isin(hpa_idcs_w_masks)]
# flt3_metadata

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,imputation_split,protcl_denovo_split,prot_denovo_split,image_id
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,ENSG00000066455,PFA,Supported,Supported,7.0,NaN,train,validation,test,1_A1_1
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,ENSG00000066455,PFA,Supported,Supported,6.0,NaN,train,validation,test,1_A1_2
10,/archive/1/1_A3_1_,https://lims.proteinatlas.org/images/2899/1_A3_1_,1,A3,1,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,ENSG00000183092,PFA,Approved,Approved,4.0,NaN,train,train,train,1_A3_1
11,/archive/1/1_A3_2_,https://lims.proteinatlas.org/images/2899/1_A3_2_,1,A3,2,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,ENSG00000183092,PFA,Approved,Approved,6.0,NaN,train,train,train,1_A3_2
16,/archive/1/1_A6_1_,https://lims.proteinatlas.org/images/609/1_A6_1_,1,A6,1,35,Annotated / Proteinatlas,"Endoplasmic reticulum,Nuclear membrane",NaN,0.0,...,ENSG00000102119,PFA,Enhanced,Enhanced,5.0,NaN,validation,train,validation,1_A6_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045170,/archive/2172/2172_F2_2_,https://lims.proteinatlas.org/images/7191/2172...,2172,F2,2,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,ENSG00000115165,PFA,Supported,Supported,14.0,"{""Cytosol"": 0.778685898, ""Midbody"": 0.00000000...",train,train,train,2172_F2_2
1045193,/archive/2172/2172_F7_1_,https://lims.proteinatlas.org/images/66270/217...,2172,F7,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,ENSG00000133488,PFA,Approved,Approved,17.0,"{""Cytosol"": 0.807385951, ""Midbody"": 0.00000000...",train,train,train,2172_F7_1
1045194,/archive/2172/2172_F7_2_,https://lims.proteinatlas.org/images/66270/217...,2172,F7,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,ENSG00000133488,PFA,Approved,Approved,13.0,"{""Cytosol"": 0.807385951, ""Midbody"": 0.00000000...",train,train,train,2172_F7_2
1045277,/archive/2172/2172_H8_3_,https://lims.proteinatlas.org/images/30507/217...,2172,H8,3,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,ENSG00000131171,PFA,Supported,Supported,13.0,"{""Cytosol"": 0.447071254, ""Midbody"": 0.00000000...",train,validation,train,2172_H8_3


# Create a protein-cell-line-pair imputation data split

In [14]:
len(v23_metadata[v23_metadata["locations"].notnull()])

78028

In [15]:
comb2img_idcs, prot2img_idcs = defaultdict(list), defaultdict(list)
for i, x in v23_metadata.iterrows():
    prot, cl = x["gene_names"], x['atlas_name']
    assert type(prot) != float
    comb2img_idcs[(prot, cl)].append(i)
    prot2img_idcs[prot].append(i)
len(comb2img_idcs), len(prot2img_idcs)

(36317, 12495)

In [16]:
Counter(map(len, comb2img_idcs.values()))

Counter({2: 31230, 8: 34, 4: 4584, 6: 343, 5: 11, 1: 72, 3: 34, 10: 9})

In [17]:
np.random.seed(42)

train_idcs, valid_idcs, test_idcs = [], [], []
for imgidcs in comb2img_idcs.values():
    n_imgs = len(imgidcs)
    if n_imgs < 4:
        train_idcs.extend(imgidcs)
    else:
        np.random.shuffle(imgidcs)
        train_idcs.extend(imgidcs[:int(0.6 * n_imgs)])
        valid_idcs.extend(imgidcs[int(0.6 * n_imgs):int(0.8 * n_imgs)])
        test_idcs.extend(imgidcs[int(0.8 * n_imgs):])
len(train_idcs), len(valid_idcs), len(test_idcs)

(73054, 5024, 5367)

In [18]:
# del metadata["imputation_split"]
# metadata.loc[train_idcs, "imputation_split"] = "train"
# metadata.loc[valid_idcs, "imputation_split"] = "validation"
# metadata.loc[test_idcs, "imputation_split"] = "test"
# metadata

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,image_id,protcl_denovo_split,prot_denovo_split,ImageHeight,ImageWidth,imputation_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,7.0,NaN,1_A1_1,test,train,1728.0,1728.0,train
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,6.0,NaN,1_A1_2,test,train,1728.0,1728.0,train
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,5.0,NaN,1_A10_1,NaN,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,4.0,NaN,1_A10_2,NaN,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,NaN,3.0,NaN,1_A11_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_20,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_21,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_22,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_23,NaN,NaN,NaN,NaN,NaN


In [19]:
max(train_idcs), max(valid_idcs), max(test_idcs)

(1045278, 1045119, 1009359)

# Create a protein-cell-line-pair denovo data split

In [20]:
all_combs = list(comb2img_idcs.keys())
np.random.shuffle(all_combs)
num_combs = len(all_combs)
train_combs = all_combs[:int(0.6 * num_combs)]
valid_combs = all_combs[int(0.6 * num_combs):int(0.8 * num_combs)]
test_combs = all_combs[int(0.8 * num_combs):]
train_prots = set([prot for (prot, cl) in train_combs])
train_cls = set([cl for (prot, cl) in train_combs])
flt_valid_combs = []
for (prot, cl) in valid_combs:
    if prot not in train_prots or cl not in train_cls:
        train_combs.append((prot, cl))
        train_prots.add(prot)
        train_cls.add(cl)
    else:
        flt_valid_combs.append((prot, cl))

flt_test_combs = []
for (prot, cl) in test_combs:
    if prot not in train_prots or cl not in train_cls:
        train_combs.append((prot, cl))
        train_prots.add(prot)
        train_cls.add(cl)
    else:
        flt_test_combs.append((prot, cl))
len(all_combs), len(train_combs), len(flt_valid_combs), len(flt_test_combs)

(36317, 22913, 6410, 6994)

In [21]:
train_idcs, valid_idcs, test_idcs = [], [], []
for comb in train_combs:
    train_idcs.extend(comb2img_idcs[comb])
for comb in valid_combs:
    valid_idcs.extend(comb2img_idcs[comb])
for comb in test_combs:
    test_idcs.extend(comb2img_idcs[comb])
len(train_idcs), len(valid_idcs), len(test_idcs)

(52616, 16595, 16675)

In [22]:
# del metadata["protcl_denovo_split"]
# metadata.loc[train_idcs, "protcl_denovo_split"] = "train"
# metadata.loc[valid_idcs, "protcl_denovo_split"] = "validation"
# metadata.loc[test_idcs, "protcl_denovo_split"] = "test"
# metadata

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,image_id,prot_denovo_split,ImageHeight,ImageWidth,imputation_split,protcl_denovo_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,7.0,NaN,1_A1_1,train,1728.0,1728.0,train,train
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,6.0,NaN,1_A1_2,train,1728.0,1728.0,train,train
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,5.0,NaN,1_A10_1,NaN,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,4.0,NaN,1_A10_2,NaN,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,NaN,3.0,NaN,1_A11_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_20,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_21,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_22,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_23,NaN,NaN,NaN,NaN,NaN


# Create a protein denovo data split

In [23]:
all_prots = list(prot2img_idcs.keys())
np.random.shuffle(all_prots)
num_prots = len(all_prots)
train_prots = all_prots[:int(0.6 * num_prots)]
valid_prots = all_prots[int(0.6 * num_prots):int(0.8 * num_prots)]
test_prots = all_prots[int(0.8 * num_prots):]
len(all_prots), len(train_prots), len(valid_prots), len(test_prots)

(12495, 7497, 2499, 2499)

In [24]:
train_idcs, valid_idcs, test_idcs = [], [], []
for prot in train_prots:
    train_idcs.extend(prot2img_idcs[prot])
for prot in valid_prots:
    valid_idcs.extend(prot2img_idcs[prot])
for prot in test_prots:
    test_idcs.extend(prot2img_idcs[prot])
len(train_idcs), len(valid_idcs), len(test_idcs)

(49968, 16829, 16648)

In [25]:
# del metadata["prot_denovo_split"]
# metadata.loc[train_idcs, "prot_denovo_split"] = "train"
# metadata.loc[valid_idcs, "prot_denovo_split"] = "validation"
# metadata.loc[test_idcs, "prot_denovo_split"] = "test"
# metadata

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,image_id,ImageHeight,ImageWidth,imputation_split,protcl_denovo_split,prot_denovo_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,7.0,NaN,1_A1_1,1728.0,1728.0,train,train,test
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,6.0,NaN,1_A1_2,1728.0,1728.0,train,train,test
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,5.0,NaN,1_A10_1,NaN,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,4.0,NaN,1_A10_2,NaN,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,NaN,3.0,NaN,1_A11_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_20,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_21,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_22,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_23,NaN,NaN,NaN,NaN,NaN


In [26]:
metadata["image_id"] = metadata["if_plate_id"].astype(str) + "_" + metadata["position"] + "_" + metadata["sample"].astype(str)
metadata

NameError: name 'meta_data' is not defined

In [27]:
# metadata.to_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv")